In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Dropout, GRU
from keras import optimizers
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
import string

# some parameters
BATCH_SIZE = 1024  # batch size for the network
EPOCH_NUMBER = 1  # number of epochs to train
THRESHOLD = 5  # symbols appearing fewer times will be replaced by a placeholder


In [3]:
from google.colab import files
uploaded = files.upload()

Saving test-B.tsv to test-B (2).tsv
Saving train-B.tsv to train-B (2).tsv
Saving valid-B.tsv to valid-B (2).tsv


In [4]:
train = pd.read_csv('train-B.tsv',encoding='utf8',sep='\t',error_bad_lines=False)
#train.index = train['id']
#x_train = train['comment_text']
#y_train = train.iloc[:, 2:]
train

,id,subtask_b,tweet
0,90194,TIN,@USER @USER Go home youβre drunk!!! @USER #M...
1,97670,TIN,@USER Liberals are all Kookoo !!!
2,52415,TIN,@USER was literally just talking about this lo...
3,13384,TIN,@USER Canada doesnβt need another CUCK! We a...
4,28414,TIN,@USER you are a lying corrupt traitor!!! Nobod...
...,...,...,...
3749,16705,UNT,@USER @USER No no thatβs shaq silver back l...
3750,84081,UNT,"@USER Chris, Chris, Chris.... Are you forgetti..."
3751,21033,UNT,@USER @USER oh shit you weren't joking... wtf
3752,61083,UNT,@USER βMy head is a fuckin eggβ


In [5]:
test = pd.read_csv('valid-B.tsv',encoding='utf8',sep='\t',error_bad_lines=False)
#train.index = train['id']
#x_train = train['comment_text']
#y_train = train.iloc[:, 2:]
test

,id,subtask_b,tweet
0,86426,UNT,@USER She should ask a few native Americans wh...
1,62688,UNT,"@USER Someone should'veTaken"" this piece of sh..."
2,77444,UNT,@USER @USER Oh noes! Tough shit.
3,25440,UNT,@USER @USER @USER She? To whom are you referr...
4,66349,UNT,@USER it's literally a narrative that is on pa...
...,...,...,...
641,93164,TIN,@USER @USER Everything else was ten years ago....
642,63482,TIN,@USER is advocating for conduct within bounds ...
643,87416,TIN,@USER @USER @USER @USER Liars like the Antifa ...
644,95338,TIN,@USER Sometimes I get strong vibes from people...


In [6]:
final_test_to_be_produced =  pd.read_csv('test-B.tsv',encoding='utf8',sep='\t',error_bad_lines=False)
#train.index = train['id']
#x_train = train['comment_text']
#y_train = train.iloc[:, 2:]
final_test_to_be_produced

,id,tweet
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1,60133,#NoPasaran: Unity demo to oppose the far-right...
2,83681,. . . What the fuck did he do this time?
3,65507,@USER Do you get the feeling he is kissing @US...
4,12588,@USER Nigga ware da hits at
...,...,...
235,22569,"#Antifa are mentally unstable cowards, pretend..."
236,48938,@USER @USER And Browning looked like dog shit ...
237,41438,All two of them taste like ass. URL
238,73439,#DespicableDems lie again about rifles. Dem Di...


In [7]:
final_test =  pd.read_csv('test-B.tsv',encoding='utf8',sep='\t',error_bad_lines=False)
#train.index = train['id']
#x_train = train['comment_text']
#y_train = train.iloc[:, 2:]
final_test

,id,tweet
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1,60133,#NoPasaran: Unity demo to oppose the far-right...
2,83681,. . . What the fuck did he do this time?
3,65507,@USER Do you get the feeling he is kissing @US...
4,12588,@USER Nigga ware da hits at
...,...,...
235,22569,"#Antifa are mentally unstable cowards, pretend..."
236,48938,@USER @USER And Browning looked like dog shit ...
237,41438,All two of them taste like ass. URL
238,73439,#DespicableDems lie again about rifles. Dem Di...


In [8]:
del train['id']
del test['id']
del final_test['id']

In [9]:
final_test

,tweet
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...
1,#NoPasaran: Unity demo to oppose the far-right...
2,. . . What the fuck did he do this time?
3,@USER Do you get the feeling he is kissing @US...
4,@USER Nigga ware da hits at
...,...
235,"#Antifa are mentally unstable cowards, pretend..."
236,@USER @USER And Browning looked like dog shit ...
237,All two of them taste like ass. URL
238,#DespicableDems lie again about rifles. Dem Di...


In [10]:
train.isnull().sum()

subtask_b    0
tweet        0
dtype: int64

In [11]:
train = train.dropna(how='any')

In [12]:
!pip install emoji

In [13]:
import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [14]:
def tokens(x):
    a = ' '.join(i for i in x.split() if not (i.isalpha() and len(i)==1))
    return a

In [15]:
train

,subtask_b,tweet
0,TIN,@USER @USER Go home youβre drunk!!! @USER #M...
1,TIN,@USER Liberals are all Kookoo !!!
2,TIN,@USER was literally just talking about this lo...
3,TIN,@USER Canada doesnβt need another CUCK! We a...
4,TIN,@USER you are a lying corrupt traitor!!! Nobod...
...,...,...
3749,UNT,@USER @USER No no thatβs shaq silver back l...
3750,UNT,"@USER Chris, Chris, Chris.... Are you forgetti..."
3751,UNT,@USER @USER oh shit you weren't joking... wtf
3752,UNT,@USER βMy head is a fuckin eggβ


In [16]:
#train data
new_train=[]
new_trains=[]
modified_train=[]
texts_train = train['tweet'].values # texts contrain all setences
for i in range(len(texts_train)):
    modified_train.append(tokens(texts_train[i]))
texts_train= modified_train
texts_train = [s.lower() for s in texts_train if not len(s)==1] # convert to lower case 
#texts_train = [word for word in texts_traintrain if word.isalpha()]
table = str.maketrans('', '', string.punctuation)
texts_train = [w.translate(table) for w in texts_train]
for i in range(len(texts_train)):
    a = remove_emoji(texts_train[i])
    new_train.append(a)
texts_train = new_train
for i in range(len(texts_train)):
    a = texts_train[i].split("https", 1)[0]
    new_trains.append(a)
texts_train = new_trains
texts_train = [s.strip(' ') for s in texts_train]
print(len(texts_train))
print(texts_train[:2])

3754
['user user go home youβ\x80\x99re drunk user maga trump2020 π\x9f\x91\x8aπ\x9f\x87ίπ\x9f\x87έπ\x9f\x91\x8a url', 'user liberals are all kookoo']


In [17]:
#validation data
new_train=[]
new_trains=[]
modified_train=[]
texts_test = test['tweet'].values # texts contrain all setences
for i in range(len(texts_test)):
    modified_train.append(tokens(texts_test[i]))
texts_test= modified_train
texts_test = [s.lower() for s in texts_test] # convert to lower case
table = str.maketrans('', '', string.punctuation)
texts_test = [w.translate(table) for w in texts_test]
for i in range(len(texts_test)):
    a = remove_emoji(texts_test[i])
    new_train.append(a)
texts_test = new_train
for i in range(len(texts_test)):
    a = texts_test[i].split("https", 1)[0]
    new_trains.append(a)
texts_test = new_trains
texts_test = [s.strip(' ') for s in texts_test]
print(len(texts_test))
print(texts_test[:2])

646
['user she should ask few native americans what their take on this is', 'user someone shouldvetaken this piece of shit to volcano π\x9f\x98\x82']


In [18]:
#test data
new_train=[]
new_trains=[]
modified_train=[]
texts_final_test = final_test['tweet'].values # texts contrain all setences
for i in range(len(texts_final_test)):
    modified_train.append(tokens(texts_final_test[i]))
texts_final_test= modified_train
texts_final_test = [s.lower() for s in texts_final_test] # convert to lower case 
#texts_train = [word for word in texts_traintrain if word.isalpha()]
table = str.maketrans('', '', string.punctuation)
texts_final_test = [w.translate(table) for w in texts_final_test]
for i in range(len(texts_final_test)):
    a = remove_emoji(texts_final_test[i])
    new_train.append(a)
texts_final_test = new_train
for i in range(len(texts_final_test)):
    a = texts_final_test[i].split("https", 1)[0]
    new_trains.append(a)
texts_final_test = new_trains
texts_final_test = [s.strip(' ') for s in texts_final_test]
print(len(texts_final_test))
print(texts_final_test[:2])

240
['whoisq wherestheserver dumpnike declasfisa democrats support antifa muslim brotherhood ms13 isis pedophilia child trafficking taxpayer funded abortion’s election fraud sedition and treason lockthemallup wwg1wga qanon \u2066user url', 'nopasaran unity demo to oppose the farright in london – antifa oct13 — enough is enough url']


In [19]:
final_test.insert (1, 2, 'NAN')

In [20]:

final_test

,tweet,2
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,NAN
1,#NoPasaran: Unity demo to oppose the far-right...,NAN
2,. . . What the fuck did he do this time?,NAN
3,@USER Do you get the feeling he is kissing @US...,NAN
4,@USER Nigga ware da hits at,NAN
...,...,...
235,"#Antifa are mentally unstable cowards, pretend...",NAN
236,@USER @USER And Browning looked like dog shit ...,NAN
237,All two of them taste like ass. URL,NAN
238,#DespicableDems lie again about rifles. Dem Di...,NAN


In [21]:
final_test = final_test[[2,'tweet']]

In [22]:
all_dfs = [train,test,final_test]
for df in all_dfs:
    df.columns = ['subtask_b', 'tweet']
all_dfs = pd.concat(all_dfs).reset_index(drop=True)

In [23]:
all_dfs
len(all_dfs)

4640

In [24]:
new_dataset=[]
new_datasets=[]
modified_train=[]
texts_total_dataset = all_dfs['tweet'].values # texts contrain all setences
for i in range(len(texts_total_dataset)):
    modified_train.append(tokens(texts_total_dataset[i]))
texts_total_dataset= modified_train
texts_total_dataset = [s.lower() for s in texts_total_dataset] # convert to lower case 
table = str.maketrans('', '', string.punctuation)
texts_total_dataset = [w.translate(table) for w in texts_total_dataset]
for i in range(len(texts_total_dataset)):
    a = remove_emoji(texts_total_dataset[i])
    new_dataset.append(a)
texts_total_dataset = new_dataset
for i in range(len(texts_total_dataset)):
    a = texts_total_dataset[i].split("https", 1)[0]
    new_datasets.append(a)
texts_total_dataset = new_datasets
texts_total_dataset = [s.strip(' ') for s in texts_total_dataset]
print(len(texts_total_dataset))
print(texts_total_dataset[:2])

4640
['user user go home youβ\x80\x99re drunk user maga trump2020 π\x9f\x91\x8aπ\x9f\x87ίπ\x9f\x87έπ\x9f\x91\x8a url', 'user liberals are all kookoo']


In [25]:
with open('listfile.txt', 'w') as filehandle:
    for listitem in texts_total_dataset:
        filehandle.write('%s\n' % listitem)

In [26]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [27]:
# Initialization
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
# Fitting
tk.fit_on_texts(texts_total_dataset)

In [28]:
# List the vocabulary
tk.word_index

{' ': 2,
 '0': 34,
 '1': 35,
 '2': 36,
 '3': 48,
 '4': 46,
 '5': 45,
 '6': 57,
 '7': 58,
 '8': 54,
 '9': 49,
 'UNK': 1,
 'a': 7,
 'b': 22,
 'c': 15,
 'd': 14,
 'e': 3,
 'f': 21,
 'g': 17,
 'h': 12,
 'i': 9,
 'j': 28,
 'k': 23,
 'l': 13,
 'm': 16,
 'n': 10,
 'o': 6,
 'p': 19,
 'q': 38,
 'r': 8,
 's': 4,
 't': 5,
 'u': 11,
 'v': 24,
 'w': 20,
 'x': 31,
 'y': 18,
 'z': 32,
 '\x80': 26,
 '\x81': 75,
 '\x82': 40,
 '\x83': 103,
 '\x84': 66,
 '\x86': 67,
 '\x87': 37,
 '\x88': 82,
 '\x89': 74,
 '\x8a': 79,
 '\x8b': 96,
 '\x8c': 83,
 '\x8d': 55,
 '\x8e': 76,
 '\x8f': 42,
 '\x90': 71,
 '\x91': 50,
 '\x92': 56,
 '\x93': 68,
 '\x94': 59,
 '\x95': 73,
 '\x96': 70,
 '\x97': 85,
 '\x98': 33,
 '\x99': 27,
 '\x9a': 88,
 '\x9b': 91,
 '\x9c': 44,
 '\x9d': 41,
 '\x9e': 90,
 '\x9f': 30,
 '£': 51,
 '¦': 61,
 '§': 87,
 '¨': 80,
 '«': 94,
 '¬': 78,
 '\xad': 60,
 '±': 95,
 '²': 109,
 '³': 92,
 '·': 65,
 '»': 72,
 '½': 69,
 '΄': 100,
 '΅': 101,
 'ά': 99,
 'έ': 43,
 'ή': 93,
 'ί': 47,
 'β': 25,
 'γ': 89,
 'ο': 5

In [29]:
sequences_train = tk.texts_to_sequences(texts_train)
print(texts_train[0])
print(sequences_train[0])

user user go home youβre drunk user maga trump2020 ππίπέπ url
[11, 4, 3, 8, 2, 11, 4, 3, 8, 2, 17, 6, 2, 12, 6, 16, 3, 2, 18, 6, 11, 25, 26, 27, 8, 3, 2, 14, 8, 11, 10, 23, 2, 11, 4, 3, 8, 2, 16, 7, 17, 7, 2, 5, 8, 11, 16, 19, 36, 34, 36, 34, 2, 29, 30, 50, 79, 29, 30, 37, 47, 29, 30, 37, 43, 29, 30, 50, 79, 2, 11, 8, 13]


In [30]:
sequences_test = tk.texts_to_sequences(texts_test)
print(texts_test[0])
print(sequences_test[0])

user she should ask few native americans what their take on this is
[11, 4, 3, 8, 2, 4, 12, 3, 2, 4, 12, 6, 11, 13, 14, 2, 7, 4, 23, 2, 21, 3, 20, 2, 10, 7, 5, 9, 24, 3, 2, 7, 16, 3, 8, 9, 15, 7, 10, 4, 2, 20, 12, 7, 5, 2, 5, 12, 3, 9, 8, 2, 5, 7, 23, 3, 2, 6, 10, 2, 5, 12, 9, 4, 2, 9, 4]


In [31]:
sequences_final_test = tk.texts_to_sequences(texts_final_test)
print(texts_final_test[0])
print(sequences_final_test[0])

whoisq wherestheserver dumpnike declasfisa democrats support antifa muslim brotherhood ms13 isis pedophilia child trafficking taxpayer funded abortion’s election fraud sedition and treason lockthemallup wwg1wga qanon ⁦user url
[20, 12, 6, 9, 4, 38, 2, 20, 12, 3, 8, 3, 4, 5, 12, 3, 4, 3, 8, 24, 3, 8, 2, 14, 11, 16, 19, 10, 9, 23, 3, 2, 14, 3, 15, 13, 7, 4, 21, 9, 4, 7, 2, 14, 3, 16, 6, 15, 8, 7, 5, 4, 2, 4, 11, 19, 19, 6, 8, 5, 2, 7, 10, 5, 9, 21, 7, 2, 16, 11, 4, 13, 9, 16, 2, 22, 8, 6, 5, 12, 3, 8, 12, 6, 6, 14, 2, 16, 4, 35, 48, 2, 9, 4, 9, 4, 2, 19, 3, 14, 6, 19, 12, 9, 13, 9, 7, 2, 15, 12, 9, 13, 14, 2, 5, 8, 7, 21, 21, 9, 15, 23, 9, 10, 17, 2, 5, 7, 31, 19, 7, 18, 3, 8, 2, 21, 11, 10, 14, 3, 14, 2, 7, 22, 6, 8, 5, 9, 6, 10, 53, 4, 2, 3, 13, 3, 15, 5, 9, 6, 10, 2, 21, 8, 7, 11, 14, 2, 4, 3, 14, 9, 5, 9, 6, 10, 2, 7, 10, 14, 2, 5, 8, 3, 7, 4, 6, 10, 2, 13, 6, 15, 23, 5, 12, 3, 16, 7, 13, 13, 11, 19, 2, 20, 20, 17, 35, 20, 17, 7, 2, 38, 7, 10, 6, 10, 2, 110, 11, 4, 3, 8, 2, 11, 8, 13

In [32]:
sequences_total_dataset = tk.texts_to_sequences(texts_total_dataset)
print(texts_total_dataset[0])
print(sequences_total_dataset[0])

user user go home youβre drunk user maga trump2020 ππίπέπ url
[11, 4, 3, 8, 2, 11, 4, 3, 8, 2, 17, 6, 2, 12, 6, 16, 3, 2, 18, 6, 11, 25, 26, 27, 8, 3, 2, 14, 8, 11, 10, 23, 2, 11, 4, 3, 8, 2, 16, 7, 17, 7, 2, 5, 8, 11, 16, 19, 36, 34, 36, 34, 2, 29, 30, 50, 79, 29, 30, 37, 47, 29, 30, 37, 43, 29, 30, 50, 79, 2, 11, 8, 13]


In [33]:
# List top 5 sentence length
for i, s in enumerate(sequences_total_dataset):
    print(len(s))
    if i > 4:
        break

73
28
176
131
100
64


In [34]:
lens=[]
for i, s in enumerate(sequences_total_dataset):
    l=len(s)
    lens.append(l)
    

In [35]:
#To get the maximum length sentence which will be used to pad the sequence
max(lens)

526

In [36]:
data_train = pad_sequences(sequences_train, maxlen=2165, padding='post')
data_test = pad_sequences(sequences_test, maxlen=2165, padding='post')
data_final_test = pad_sequences(sequences_final_test, maxlen=2165, padding='post')

In [37]:
data_train = np.array(data_train)
data_train.shape # All sentence length are 1014

(3754, 2165)

In [38]:
data_final_test = np.array(data_final_test)
data_final_test.shape # All sentence length are 1014

(240, 2165)

In [39]:
data_test = np.array(data_test)
data_test.shape # All sentence length are 1014

(646, 2165)

In [40]:
train['subtask_b'].unique()

array(['TIN', 'UNT'], dtype=object)

In [41]:
train['subtask_b'] == 'TIN'

0        True
1        True
2        True
3        True
4        True
        ...  
3749    False
3750    False
3751    False
3752    False
3753    False
Name: subtask_b, Length: 3754, dtype: bool

In [42]:
print(train['subtask_b'].unique()) # how many unique classes
train['subtask_b'] = np.where(train['subtask_b'] == 'UNT', 0, 1)
print(train['subtask_b'].unique()) # how many unique classes
class_list = train['subtask_b'].values
print(len(class_list)) # how many length
print(class_list[:10]) # print top 10 samples classes

['TIN' 'UNT']
[1 0]
3754
[1 1 1 1 1 1 1 1 1 1]


In [43]:
print(test['subtask_b'].unique()) # how many unique classes
test['subtask_b'] = np.where(test['subtask_b'] == 'UNT', 0, 1)
print(test['subtask_b'].unique()) # how many unique classes
class_list_test = test['subtask_b'].values
print(len(class_list_test)) # how many length
print(class_list_test[:10]) # print top 10 samples classes

['UNT' 'TIN']
[0 1]
646
[0 0 0 0 0 0 0 0 0 0]


In [44]:
print(sequences_train[0][:160])
print()
print(data_train[0][:160])

[11, 4, 3, 8, 2, 11, 4, 3, 8, 2, 17, 6, 2, 12, 6, 16, 3, 2, 18, 6, 11, 25, 26, 27, 8, 3, 2, 14, 8, 11, 10, 23, 2, 11, 4, 3, 8, 2, 16, 7, 17, 7, 2, 5, 8, 11, 16, 19, 36, 34, 36, 34, 2, 29, 30, 50, 79, 29, 30, 37, 47, 29, 30, 37, 43, 29, 30, 50, 79, 2, 11, 8, 13]

[11  4  3  8  2 11  4  3  8  2 17  6  2 12  6 16  3  2 18  6 11 25 26 27
  8  3  2 14  8 11 10 23  2 11  4  3  8  2 16  7 17  7  2  5  8 11 16 19
 36 34 36 34  2 29 30 50 79 29 30 37 47 29 30 37 43 29 30 50 79  2 11  8
 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [45]:
from keras.utils.np_utils import to_categorical
classes = to_categorical(class_list)
print(class_list[:10])
print(classes[:10])

[1 1 1 1 1 1 1 1 1 1]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [46]:
from keras.utils.np_utils import to_categorical
classes_test = to_categorical(class_list_test)
print(class_list_test[:10])
print(classes_test[:10])

[0 0 0 0 0 0 0 0 0 0]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [47]:
x_train, x_test, y_train, y_test = data_train,data_test,classes,classes_test

In [48]:
x_train.shape
y_train.shape

(3754, 2)

In [49]:
vocab_size = len(tk.word_index)
vocab_size

110

In [50]:
embedding_weights = [] #(71, 70)
embedding_weights.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [51]:
embedding_weights.shape

(111, 110)

In [63]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout,LSTM,Conv2D
from keras.models import Model
from keras.layers import Bidirectional

In [64]:
# parameter 
input_size = 2165
# vocab_size = 69
embedding_size = 110
conv_layers = [[128, 7, 3],
               [128, 7, 3], 
              [128, 3, -1], 
               [128, 3, -1]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.4
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [65]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights],trainable=True)

In [66]:
#three# Attention GRU network		 
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        #print(input_shape)
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        #self.trainable_weights = [self.W]
        self.W = self.add_weight(name='W', 
                                      shape=(input_shape[-1],),
                                      initializer='uniform',
                                      trainable=True)
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1))
        #eij=K.tanh(K.dot(x, K.expand_dims(self.W)))
        print(eij.shape)
        
        ai = K.exp(eij)
        weights = ai/K.expand_dims(K.sum(ai, axis=1),1)#ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input =  x*K.expand_dims(weights,2)#x*weights.dimshuffle(0,1,'x')
        print(weighted_input.shape)
        sum_1 = K.sum(weighted_input, axis=1)
#         print(sum_1.shape)
        return sum_1#weighted_input.sum(axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [67]:
from keras.layers.merge import concatenate
from keras import regularizers
sequence_input = Input(shape=(input_size,), dtype='int32')
#print(sequence_input.shape)
embedded_sequences = embedding_layer(sequence_input)
conv = Conv1D(filters=32, kernel_size=3, activation='relu')(embedded_sequences)
maxp = MaxPooling1D(pool_size=3)(conv)
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(maxp)
maxp1 = MaxPooling1D(pool_size=3)(conv1)
#conv2 = Conv1D(filters=32, kernel_size=3, activation='relu')(maxp1)
#print(embedded_sequences.shape)
l_gru = Bidirectional(LSTM(20))(maxp1)
l_gru = LSTM(100,return_sequences=True)(maxp1)
#print(l_gru.shape)
l_att = AttLayer()(l_gru)
#l_gru1 = Bidirectional(LSTM(100, return_sequences=True))(embedded_sequences)
#l_att1 = AttLayer()(l_gru1)
#conca = concatenate([l_att, l_att1])
preds = Dense(32, activation='tanh')(l_att)
preds = Dropout(0.5)(preds)
predss=Dense(2, activation='softmax')(preds)
#print(predss.shape)
model = Model(inputs=sequence_input, outputs=predss)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.summary()

(None, 239)
(None, 239, 100)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2165)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 2165, 110)         12210     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2163, 32)          10592     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 721, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 719, 32)           3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 239, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (

In [80]:
class_weight = {0: 80.,
                1: 10.}

In [81]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=10,
          epochs=6,
          verbose=1,
          class_weight=class_weight)

Epoch 1/6
376/376 [==============================] - 7s 19ms/step - loss: 9.6243 - acc: 0.7619 - val_loss: 0.6089 - val_acc: 0.6811
Epoch 2/6
376/376 [==============================] - 7s 18ms/step - loss: 9.2188 - acc: 0.7834 - val_loss: 0.5737 - val_acc: 0.7276
Epoch 3/6
376/376 [==============================] - 7s 18ms/step - loss: 9.0753 - acc: 0.7592 - val_loss: 0.5982 - val_acc: 0.6780
Epoch 4/6
376/376 [==============================] - 7s 18ms/step - loss: 8.5731 - acc: 0.7874 - val_loss: 0.6347 - val_acc: 0.6796
Epoch 5/6
376/376 [==============================] - 7s 18ms/step - loss: 8.2127 - acc: 0.7680 - val_loss: 0.7098 - val_acc: 0.6471
Epoch 6/6
376/376 [==============================] - 7s 18ms/step - loss: 7.7287 - acc: 0.7981 - val_loss: 0.6568 - val_acc: 0.7028


In [82]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [83]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score
def evaluation(): 
    # predict class with test set
    y_pred_test =  model.predict(x_test, batch_size=10, verbose=0)
    class_list=[]
    for i in range(len(y_pred_test)):
        class_list.append(np.argmax(y_pred_test[i]))
    y_pred_test = np.asarray(class_list)
    print('Accuracy:\t{:0.1f}%'.format(accuracy_score(np.argmax(y_test,axis=1),y_pred_test)*100))
    
    #classification report
    print('\n')
    print(classification_report(np.argmax(y_test,axis=1), y_pred_test))

    #confusion matrix
    confmat = confusion_matrix(np.argmax(y_test,axis=1), y_pred_test)
    print("The Confusion matrix is: -")
    print(confmat)
    return y_pred_test

In [84]:
#This code is written to call the function which will evalute the test data based on the system
y_pred = evaluation()
#print(y_pred)

Accuracy:	70.3%


              precision    recall  f1-score   support

           0       0.39      0.26      0.31       168
           1       0.77      0.86      0.81       478

    accuracy                           0.70       646
   macro avg       0.58      0.56      0.56       646
weighted avg       0.67      0.70      0.68       646

The Confusion matrix is: -
[[ 43 125]
 [ 67 411]]


In [92]:
test

,subtask_a,tweet
0,0,@USER Gun control is omportant. It should not...
1,0,@USER Antifa girl of the month centrefold!
2,1,@USER @USER @USER @USER Tweet is directed at h...
3,0,@USER Oh well yes goes without saying you are ...
4,0,@USER Good!!! MAGA
...,...,...
2636,1,@USER Sometimes I get strong vibes from people...
2637,0,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...
2638,1,@USER And why report this garbage. We don't g...
2639,1,@USER Pussy


In [93]:
x_test

array([[11,  4,  3, ...,  0,  0,  0],
       [11,  4,  3, ...,  0,  0,  0],
       [11,  4,  3, ...,  0,  0,  0],
       ...,
       [11,  4,  3, ...,  0,  0,  0],
       [11,  4,  3, ...,  0,  0,  0],
       [ 4, 20,  8, ...,  0,  0,  0]], dtype=int32)

In [94]:
y_final_pred = model.predict(data_final_test, batch_size=10, verbose=0)

(10, 239)
(10, 239, 100)


In [95]:
class_list=[]
for i in range(len(y_final_pred)):
        class_list.append(np.argmax(y_final_pred[i]))
y_final_pred = np.asarray(class_list)

In [96]:
y_final_pred[0:10]

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [97]:
len(final_test_to_be_produced)

860

In [98]:
y_final_pred = y_final_pred.tolist()

In [99]:
del final_test_to_be_produced['tweet']

In [100]:
final_test_to_be_produced['label'] = y_final_pred

In [101]:
final_test_to_be_produced

,id,label
0,15923,1
1,27014,0
2,30530,0
3,13876,0
4,60133,0
...,...,...
855,73439,1
856,25657,0
857,67018,0
858,50665,1


In [102]:
final_test_to_be_produced['label']=np.where(final_test_to_be_produced['label'] == 0,'NOT', 'OFF')

In [103]:
final_test_to_be_produced

,id,label
0,15923,OFF
1,27014,NOT
2,30530,NOT
3,13876,NOT
4,60133,NOT
...,...,...
855,73439,OFF
856,25657,NOT
857,67018,NOT
858,50665,OFF


# New section

In [104]:
final_test_to_be_produced.to_csv('Task-NLPweek-a.tsv', index=None, sep='\t', mode='a')